# Ran in Google Colab

# Loading Text File from Kaggle into Dataframe

In [1]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp

openjdk version "1.8.0_312"
OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~18.04-b07)
OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)
  Using cached pyspark-2.4.4-py2.py3-none-any.whl
  Using cached py4j-0.10.7-py2.py3-none-any.whl (197 kB)
  Using cached spark_nlp-3.3.4-py2.py3-none-any.whl (133 kB)


In [3]:
! pip install -q kaggle

In [47]:
from google.colab import files
files.upload()

{}

In [5]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              237KB  2021-12-12 11:59:54          17996  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01          10722  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           5353  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-10-22 10:48:21           3845  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [8]:
! kaggle datasets download -d uciml/electric-power-consumption-data-set

electric-power-consumption-data-set.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
! unzip electric-power-consumption-data-set.zip

Archive:  electric-power-consumption-data-set.zip
replace household_power_consumption.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: household_power_consumption.txt  


In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder.getOrCreate()

In [12]:
df = spark.read.text('household_power_consumption.txt')

In [13]:
df.show(5)

+--------------------+
|               value|
+--------------------+
|Date;Time;Global_...|
|16/12/2006;17:24:...|
|16/12/2006;17:25:...|
|16/12/2006;17:26:...|
|16/12/2006;17:27:...|
+--------------------+
only showing top 5 rows



In [14]:
dfColumnsNotSplit = df.selectExpr("split(value, ';')\
as Text_Data_In_A_Single_Column")

dfColumnsNotSplit.show(5,False)

+-----------------------------------------------------------------------------------------------------------------------------------+
|Text_Data_In_A_Single_Column                                                                                                       |
+-----------------------------------------------------------------------------------------------------------------------------------+
|[Date, Time, Global_active_power, Global_reactive_power, Voltage, Global_intensity, Sub_metering_1, Sub_metering_2, Sub_metering_3]|
|[16/12/2006, 17:24:00, 4.216, 0.418, 234.840, 18.400, 0.000, 1.000, 17.000]                                                        |
|[16/12/2006, 17:25:00, 5.360, 0.436, 233.630, 23.000, 0.000, 1.000, 16.000]                                                        |
|[16/12/2006, 17:26:00, 5.374, 0.498, 233.290, 23.000, 0.000, 2.000, 17.000]                                                        |
|[16/12/2006, 17:27:00, 5.388, 0.502, 233.740, 23.000, 0.000, 

In [15]:
dfColumnsSplit = spark.read.option("header", "true") \
    .option("delimiter", ";") \
    .option("inferSchema", "true") \
    .csv("household_power_consumption.txt")

dfColumnsSplit.show(5, truncate=False)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|Date      |Time    |Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|16/12/2006|17:24:00|4.216              |0.418                |234.840|18.400          |0.000         |1.000         |17.0          |
|16/12/2006|17:25:00|5.360              |0.436                |233.630|23.000          |0.000         |1.000         |16.0          |
|16/12/2006|17:26:00|5.374              |0.498                |233.290|23.000          |0.000         |2.000         |17.0          |
|16/12/2006|17:27:00|5.388              |0.502                |233.740|23.000          |0.000         |1.000         |17.0          |
|16/12/2006|17:28:00|3.666              |0.528                

# Select Query

## Show a single column

In [16]:
dfColumnsSplit.select("Global_active_power").show(5) 

+-------------------+
|Global_active_power|
+-------------------+
|              4.216|
|              5.360|
|              5.374|
|              5.388|
|              3.666|
+-------------------+
only showing top 5 rows



## Showing multiple columns

In [17]:
dfColumnsSplit.select("Global_active_power", "Global_reactive_power", "Voltage", "Global_intensity").show(5)

+-------------------+---------------------+-------+----------------+
|Global_active_power|Global_reactive_power|Voltage|Global_intensity|
+-------------------+---------------------+-------+----------------+
|              4.216|                0.418|234.840|          18.400|
|              5.360|                0.436|233.630|          23.000|
|              5.374|                0.498|233.290|          23.000|
|              5.388|                0.502|233.740|          23.000|
|              3.666|                0.528|235.680|          15.800|
+-------------------+---------------------+-------+----------------+
only showing top 5 rows



# Like Query

In [18]:
dfColumnsSplit.select("Global_active_power", dfColumnsSplit.Global_active_power.like("5.388")).show(5)

+-------------------+------------------------------+
|Global_active_power|Global_active_power LIKE 5.388|
+-------------------+------------------------------+
|              4.216|                         false|
|              5.360|                         false|
|              5.374|                         false|
|              5.388|                          true|
|              3.666|                         false|
+-------------------+------------------------------+
only showing top 5 rows



# Filter Query

In [19]:
dfColumnsSplit.filter \
    (dfColumnsSplit.Global_active_power == "5.388").show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|16/12/2006|17:27:00|              5.388|                0.502|233.740|          23.000|         0.000|         1.000|          17.0|
|22/12/2006|20:53:00|              5.388|                0.252|232.360|          23.200|        36.000|         1.000|          17.0|
|24/12/2006|00:04:00|              5.388|                0.000|236.980|          22.600|         0.000|         0.000|          17.0|
|28/12/2006|19:00:00|              5.388|                0.176|234.950|          22.800|         0.000|        38.000|          17.0|
|28/12/2006|19:07:00|              5.388|                0.182

# Group By Query

In [20]:
dfColumnsSplit.groupBy("Date").count().show(5)

+----------+-----+
|      Date|count|
+----------+-----+
| 30/1/2007| 1440|
| 13/2/2007| 1440|
|19/11/2007| 1440|
| 12/1/2008| 1440|
| 26/2/2008| 1440|
+----------+-----+
only showing top 5 rows



# Handling NaNs 

## Finds NaNs

In [21]:
dfColumnsSplit.filter("Sub_metering_3 is NULL").show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|21/12/2006|11:23:00|                  ?|                    ?|      ?|               ?|             ?|             ?|          null|
|21/12/2006|11:24:00|                  ?|                    ?|      ?|               ?|             ?|             ?|          null|
|30/12/2006|10:08:00|                  ?|                    ?|      ?|               ?|             ?|             ?|          null|
|30/12/2006|10:09:00|                  ?|                    ?|      ?|               ?|             ?|             ?|          null|
| 14/1/2007|18:36:00|                  ?|                    ?

## Fill NaNs

In [22]:
dfColumnsNaNFill = dfColumnsSplit.na.fill(123456)

In [23]:
dfColumnsNaNFill.filter \
    (dfColumnsNaNFill.Sub_metering_3 == "123456").show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|21/12/2006|11:23:00|                  ?|                    ?|      ?|               ?|             ?|             ?|      123456.0|
|21/12/2006|11:24:00|                  ?|                    ?|      ?|               ?|             ?|             ?|      123456.0|
|30/12/2006|10:08:00|                  ?|                    ?|      ?|               ?|             ?|             ?|      123456.0|
|30/12/2006|10:09:00|                  ?|                    ?|      ?|               ?|             ?|             ?|      123456.0|
| 14/1/2007|18:36:00|                  ?|                    ?

# Modify Table - Columns

## Update Query - One Column's value


In [24]:
from pyspark.sql import functions as F

In [25]:
dfColumnsFill_OneColumn = dfColumnsNaNFill.withColumn \
    ("Global_active_power", F.when(F.col("Global_active_power")=='?', 100).otherwise(F.col("Global_active_power")))


In [26]:
dfColumnsFill_OneColumn.filter \
    (dfColumnsFill_OneColumn.Sub_metering_3 == "123456").show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|21/12/2006|11:23:00|                100|                    ?|      ?|               ?|             ?|             ?|      123456.0|
|21/12/2006|11:24:00|                100|                    ?|      ?|               ?|             ?|             ?|      123456.0|
|30/12/2006|10:08:00|                100|                    ?|      ?|               ?|             ?|             ?|      123456.0|
|30/12/2006|10:09:00|                100|                    ?|      ?|               ?|             ?|             ?|      123456.0|
| 14/1/2007|18:36:00|                100|                    ?

## Update Query - All Columns value


In [27]:
cols = dfColumnsNaNFill.columns
for col in cols:
  dfColumnsNaNFill = dfColumnsNaNFill.withColumn(col, F.when(dfColumnsNaNFill[col] == "?",100).otherwise(F.col(col)))

In [28]:
dfColumnsNaNFill.filter \
    (dfColumnsFill_OneColumn.Sub_metering_3 == "123456").show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|21/12/2006|11:23:00|                100|                  100|    100|             100|           100|           100|      123456.0|
|21/12/2006|11:24:00|                100|                  100|    100|             100|           100|           100|      123456.0|
|30/12/2006|10:08:00|                100|                  100|    100|             100|           100|           100|      123456.0|
|30/12/2006|10:09:00|                100|                  100|    100|             100|           100|           100|      123456.0|
| 14/1/2007|18:36:00|                100|                  100

## Update Column Name

In [29]:
dfColumnsSplit_ColumnRename = dfColumnsSplit.withColumnRenamed('Global_active_power', 'Global Active Power')

dfColumnsSplit_ColumnRename.show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global Active Power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|16/12/2006|17:24:00|              4.216|                0.418|234.840|          18.400|         0.000|         1.000|          17.0|
|16/12/2006|17:25:00|              5.360|                0.436|233.630|          23.000|         0.000|         1.000|          16.0|
|16/12/2006|17:26:00|              5.374|                0.498|233.290|          23.000|         0.000|         2.000|          17.0|
|16/12/2006|17:27:00|              5.388|                0.502|233.740|          23.000|         0.000|         1.000|          17.0|
|16/12/2006|17:28:00|              3.666|                0.528

## Add Query

In [30]:
dfColumnsSplit_NewColumn = dfColumnsSplit.withColumn('new_column', F.lit('#66DaysOfData'))

dfColumnsSplit_NewColumn.show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+-------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|   new_column|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+-------------+
|16/12/2006|17:24:00|              4.216|                0.418|234.840|          18.400|         0.000|         1.000|          17.0|#66DaysOfData|
|16/12/2006|17:25:00|              5.360|                0.436|233.630|          23.000|         0.000|         1.000|          16.0|#66DaysOfData|
|16/12/2006|17:26:00|              5.374|                0.498|233.290|          23.000|         0.000|         2.000|          17.0|#66DaysOfData|
|16/12/2006|17:27:00|              5.388|                0.502|233.740|          23.000|         0.000|         

## Delete Query

In [31]:
dfColumnsSplit_DeleteColumn = dfColumnsSplit.drop("Global_active_power")

dfColumnsSplit_DeleteColumn.show(5)

+----------+--------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+---------------------+-------+----------------+--------------+--------------+--------------+
|16/12/2006|17:24:00|                0.418|234.840|          18.400|         0.000|         1.000|          17.0|
|16/12/2006|17:25:00|                0.436|233.630|          23.000|         0.000|         1.000|          16.0|
|16/12/2006|17:26:00|                0.498|233.290|          23.000|         0.000|         2.000|          17.0|
|16/12/2006|17:27:00|                0.502|233.740|          23.000|         0.000|         1.000|          17.0|
|16/12/2006|17:28:00|                0.528|235.680|          15.800|         0.000|         1.000|          17.0|
+----------+--------+---------------------+-------+----------------+--------------+-----

# Modify Table - Rows

## Add Row Query

In [32]:
newRow = spark.createDataFrame([('21/12/2021','17:24:00',4.216,0.418,234.840,18.400,0.00,0.01,17.0)], dfColumnsSplit.columns)

dfColumnsSplit_NewRow = dfColumnsSplit.union(newRow)

In [33]:
dfColumnsSplit_NewRow.filter \
    (dfColumnsSplit_NewRow.Date == '21/12/2021').show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|21/12/2021|17:24:00|              4.216|                0.418| 234.84|            18.4|           0.0|          0.01|          17.0|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+



## Update Row Query

In [34]:
dfColumnsSplit_UpdateRow = dfColumnsSplit_NewRow.withColumn(
    "Sub_metering_1",
    F.when(
        F.col("Date") == '21/12/2021',
        15.0
    ).otherwise(F.col("Sub_metering_1"))
)

In [35]:
dfColumnsSplit_UpdateRow.filter \
    (dfColumnsSplit_UpdateRow.Date == '21/12/2021').show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|21/12/2021|17:24:00|              4.216|                0.418| 234.84|            18.4|          15.0|          0.01|          17.0|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+



## Drop Row

In [36]:
dfColumnsSplit_DropRow = dfColumnsSplit_NewRow.where("Date == 21/12/2021")

In [37]:
dfColumnsSplit_DropRow.filter \
    (dfColumnsSplit_DropRow.Date == '21/12/2021').show(5)

+----+----+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|Date|Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----+----+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
+----+----+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+



## Drop NaNs

In [38]:
dfColumnsSplit.count()

2075259

In [45]:
dfColumnsSplit_DropNaNsAll = dfColumnsSplit.na.drop("all")

dfColumnsSplit_DropNaNsAll.count()

2075259

In [41]:
dfColumnsSplit_DropNaNsAny = dfColumnsSplit.na.drop("any")

dfColumnsSplit_DropNaNsAny.count()

2049280

# Registering Tables with SQL

In [42]:
dfColumnsSplit.registerTempTable("df")

In [43]:
spark.sql("SELECT * FROM df").show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|16/12/2006|17:24:00|              4.216|                0.418|234.840|          18.400|         0.000|         1.000|          17.0|
|16/12/2006|17:25:00|              5.360|                0.436|233.630|          23.000|         0.000|         1.000|          16.0|
|16/12/2006|17:26:00|              5.374|                0.498|233.290|          23.000|         0.000|         2.000|          17.0|
|16/12/2006|17:27:00|              5.388|                0.502|233.740|          23.000|         0.000|         1.000|          17.0|
|16/12/2006|17:28:00|              3.666|                0.528

# Stop Session

In [46]:
spark.stop()